### dependence ###

In [1]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MeanShift
from sklearn.cluster import hierarchical
from sklearn.cluster import estimate_bandwidth
from sklearn.metrics import calinski_harabaz_score
from sklearn.metrics import silhouette_score
from jieba.analyse import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# from Bio.Cluster import kcluster
# import synonyms
import pandas as pd
import jieba
import numpy as np
import re
import jieba.posseg as pseg

### other function ###

In [2]:
n_clusters = 13

def deal_sklearn_model(model,
                       weight,
                       data,
                       df,
                       X,
                       print_details=0,
                       distance='euclidean'):
    kind = df[1].tolist()
    if print_details:
        print(model.labels_)
        print(len(model.labels_))
        details = {}
        for i in range(len(data)):
            if model.labels_[i] not in details:
                details[model.labels_[i]] = [[data[i]], [kind[i]]]
            else:
                details[model.labels_[i]][0].append(data[i])
                details[model.labels_[i]][1].append(kind[i])
        number = 0
        for key, list in details.items():
            this_kind = 0
            print("第%d类" % key)
            for i in range(len(list[0])):
                this_kind = this_kind + 1
                print("属于", list[1][i], ' ', list[0][i])
            print(this_kind)
            print()
            print()
            print()
            number += this_kind
        print("all_number == ", number)
    check_result(weight, model, distance)


    # K-Means 算法
def K_Means(weight, data, df, X, print_details=0):
    print("K_Means算法")
    K_number = 16
    iter_number = 700
    kmodel = KMeans(n_clusters=n_clusters,
                    n_init=10,
                    max_iter=iter_number,
                    init='k-means++',
                   n_jobs=-1).fit(weight)
    deal_sklearn_model(kmodel, weight, data, df, X, print_details)
    return kmodel.labels_


# DBSCAN 算法
def AAedion_DBSCAN(weight, data, df, X, print_details=0, distance='euclidean'):
    print("DBSCAN算法")
    DBmodel = DBSCAN(eps=0.6, min_samples=5, metric=distance,n_jobs=-1).fit(weight)
    deal_sklearn_model(DBmodel, weight, data, df, X, print_details, distance)
    print("DBSCAN算法完成")
    return DBmodel.labels_


def AAedion_Birch(weight, data, df, X, print_details=0, distance='euclidean'):
    print('Birch算法')
    Birch_model = Birch(n_clusters=n_clusters).fit(weight)
    deal_sklearn_model(
        Birch_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return Birch_model.labels_


def AAedion_SpectralClustering(weight,
                               data,
                               df,
                               X,
                               print_details=0,
                               distance='euclidean',n_jobs=-1):
    print('SpectralClustering算法')
    Spe_model = SpectralClustering(n_clusters=n_clusters).fit(weight)
    deal_sklearn_model(
        Spe_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return Spe_model.labels_


def AAedion_AffinityPropagation(weight,
                                data,
                                df,
                                X,
                                print_details=0,
                                distance='euclidean'):
    print('AffinityPropagatio算法')
    Aff_model = AffinityPropagation(damping=0.5,
                                    max_iter=200,
                                    convergence_iter=15,
                                    copy=True,
                                    preference=None,
                                    affinity='euclidean',
                                    verbose=False).fit(weight)
    deal_sklearn_model(
        Aff_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return Aff_model.labels_


def AAedion_Meanshift(weight,
                      data,
                      df,
                      X,
                      print_details=0,
                      distance='euclidean'):
    print('MeanShift算法')
#     bandwidth = estimate_bandwidth(weight, quantile=0.3,n_jobs=-1)
#    print(bandwidth)
    MeanShift_model = MeanShift(bandwidth=1,
                                seeds=None,
                                bin_seeding=True,
                                min_bin_freq=1,
                                cluster_all=False,
                                n_jobs=-1).fit(weight)
    print('Meanshfit finish')
    deal_sklearn_model(
        MeanShift_model,
        weight,
        data,
        df,
        X,
        print_details,
    )
    return MeanShift_model.labels_


def check_result(weight, model, distance):
    #print("共%d类" %model.labels_)
    print("result of harabaz_score: ",
          calinski_harabaz_score(weight, model.labels_))
    print("result of silhouette_score: ",
          silhouette_score(weight, model.labels_, metric='euclidean'))


def print_TF_IDF_values():
    for i in range(len(weight)):  # 打印每类文本的tf-idf词语权重
        print("-------这里输出第", i, u"类文本的词语tf-idf权重------")
        # for j in range(len(word)):
        print(list(zip(all_word, weight[i])))

### stop word ###

In [3]:
    # 人工停用词表
    special_word = ['称', '，', '。', '在', '年', '月', '(', '（','民警','报警','观音桥']

    # 加载停用词表
    Stopword_FilePath = "../stopWord.txt"
    stopWordList = pd.read_csv(Stopword_FilePath, sep="\r")
    for word in special_word:
        stopWordList.loc[stopWordList.size] = word
    print("停用词表的大小是", stopWordList.size)

    # 加载数据
    FilePath = "../data.csv"
    df = pd.read_csv(FilePath,header=None)

    # 停用词过滤
    data = df[5]
    X = []
    delete = []
    record = {}
    person_number = 0
    place_number = 0
    pronoun_number = 0
    beside_number = 0
    all_number = 0
    flag_record = {}
    flag_form = {}
    flag_result = {}
    data_keywords = []
    for i in range(len(data)):
        cut_result = []
        person_name = []
        place_name = []
        pronoun = []
        beside = []
        # 按词性过滤
        k = pseg.cut(data[i])
        for word, flag in k:
            flag_form[word] = flag
            if flag not in flag_record:
                flag_record[flag] = 1
            else:
                flag_record[flag] += 1

            if flag == 'nr' or flag == 'nr1' or flag == 'nr2':
                person_name.append(word)
                person_number += 1
                continue
            if flag == 'ns' or flag == 'nz':
                place_name.append(word)
                place_number += 1
                continue
            if flag == 'r' or flag == 'rr' or flag == 'rz':
                pronoun.append(word)
                pronoun_number += 1
                continue
            if flag == 'w' or flag == 'h' or flag == 'k' or flag == 'xx' \
                    or flag == 'o' or flag == 'u' or flag == 'm' or flag == 'd' \
                    or flag == 'f':
                beside.append(word)
                beside_number += 1
                continue
        # 提取关键词列表
        for keyword, weight in extract_tags(data[i], withWeight=True):
            if keyword in person_name or keyword in place_name or keyword in pronoun or keyword in beside:
                all_number += 1
                delete.append(keyword)
                continue
            # if flag_form[keyword] not in flag_result:
            #     flag_result[flag_form[keyword]] = [keyword]
            # else:
            #     flag_result[flag_form[keyword]].append(keyword)
            cut_result.append(keyword)
        # print(data[i],": 关键词",cut_result)

        # 用停用词库筛选关键词列表
        temp = ""
        for j in cut_result:
            if j in stopWordList.values or re.search("[0-9]+", j) is not None \
                    or re.search("[a-z]+", j) is not None or re.search("[A-Z]+", j) is not None:
                if j not in delete:
                    delete.append(j)
            else:
                if temp == '':
                    temp = j
                else:
                    temp = temp + " " + j
                if j in record:
                    record[j] = record[j] + 1
                else:
                    record[j] = 1
        X.append(temp)
        data_keywords.append(temp)
    flag_record = sorted(flag_record.items(), key=lambda item: item[1], reverse=True)
    result_X = []
    for string in X:
        temp = ''
        for j in string.split(' '):
            if j == '':
                continue
            if record[j] < 5:
                continue
            temp = temp + " " + j
            # print(j)
        result_X.append(temp)
    X_str = np.array(result_X)
    record = sorted(record.items(), key=lambda item: item[1], reverse=True)
    
#     print("被清除的词有", delete)
#     print("最终词典", record)
#     print("过滤结果", X_str)
#     print('person_number: ', person_number)
#     print('place_number: ', place_number)
#     print('pronoun_number: ', pronoun_number)
#     print('beside_number: ', beside_number)
#     print('all_number: ', all_number)
#     print('flag_record: ', flag_record)

    # for w, f in flag_result.items():
    #     print('%s类' % w)
    #     for str in f:
    #         print(str)
    #     print()
    #     print()
    #     print()

Building prefix dict from the default dictionary ...
Loading model from cache E:\local_temp\jieba.cache


停用词表的大小是 2079


Loading model cost 0.741 seconds.
Prefix dict has been built succesfully.


### construction TF-IDF ###

In [4]:
vectorizer = CountVectorizer()  # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
X = vectorizer.fit_transform(X_str)  # 将文本转为词频矩阵
tfidf = transformer.fit_transform(X)  # 计算tf-idf，
all_word = vectorizer.get_feature_names()  # 获取词袋模型中的所有词语
weight = tfidf.toarray()  # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

# print_TF_IDF_values()
print("weight shape:", weight.shape)
print("word shape:", len(all_word))

weight shape: (6481, 787)
word shape: 787


### Cluster ###

In [5]:
def clustering(print_details,weight):
    KMeans_labels = K_Means(weight, data, df, X, print_details)
    # DBSCAN_labels = AAedion_DBSCAN(weight, data, df, X, print_details)
    Birch_labels = AAedion_Birch(weight, data, df, X, print_details)
    SpectralClustering_labels = AAedion_SpectralClustering(
        weight, data, df, X, print_details)
    all_labels = [
        KMeans_labels, Birch_labels, SpectralClustering_labels
    ]
    return all_labels


# KMeans_labels = K_Means(weight, data, df, X, print_details)
# DBSCAN_labels = AAedion_DBSCAN(weight, data, df, X, print_details)
# Brich_labels = AAedion_Birch(weight, data, df, X, print_details)
# SpectralClustering_labels = AAedion_SpectralClustering(weight, data, df, X, print_details)

### can't use 

# AAedion_AffinityPropagation(weight, data, df, X, 1)
# AAedion_Meanshift(weight, data, df, X, 1)

In [6]:
def deal(all_labels):
    cluster_result = []
    for i in range(0, len(all_labels[0])):
        cluster_result.append('')
    for labels in all_labels:
        for i in range(0, len(labels)):
            cluster_result[i] = cluster_result[i] + " " + str(labels[i])
    return cluster_result


def Change(all_labels,weight):
    cluster_result = deal(all_labels)
    cluster_map = {}
    for i in range(len(cluster_result)):
        string = cluster_result[i]
        if string in cluster_map:
            cluster_map[string][0].append(data[i])
            cluster_map[string][1].append(i)
            cluster_map[string][2] += 1
        else:
            cluster_map[string] = [[data[i]], [i], 1]

    cluster_map = sorted(cluster_map.items(),
                         key=lambda item: item[1][2],
                         reverse=True)
    print("聚类总情况数: ", len(cluster_map))
    # for item in cluster_map:
    #     print("所属:  ", item[0], "数量： ", item[1][2])
    goal_words = {}
    for i in range(0, len(cluster_map)):
        if cluster_map[i][1][2] < len(data)/40:
            continue
        print(i)
        wordcount_record = {}
        for item in cluster_map[i][1][1]:
            #     print(data_keywords[item])
            for word in data_keywords[item].split(" "):
                if word == ' ' or word == '':
                    continue
                if word in wordcount_record:
                    wordcount_record[word] += 1
                else:
                    wordcount_record[word] = 1
        wordcount_record = sorted(wordcount_record.items(),
                                  key=lambda item: item[1],
                                  reverse=True)
        for item in wordcount_record[0:2]:
            print(item[0], " ", item[1])
            weight = changeWeight(weight,all_word,item[0],1.4,0)
    return weight
        

def changeWeight(weight,all_word,word,change_rate,print_details = 0):
    change_array = np.eye(weight.shape[1])
    for i in range(len(all_word)):
        if all_word[i] == word:
            break
    change_array[i][i] = change_rate
    weight = np.dot(weight, change_array)
    if print_details:
        see_change_list = []
        for k in range(0, 100):
            see_change_list.append(weight[k][i])
            for k in range(0, 100):
                print("from: ", see_change_list[k], " to: ", weight[k][i])
    return weight

In [7]:
for i in range(0, 5):
    print('第%d次' % i)
    print_details = 0
    all_labels = clustering(print_details, weight)
    weight = Change(all_labels, weight)

第0次
K_Means算法
result of harabaz_score:  166.54794680673305
result of silhouette_score:  0.17425475593696435
Birch算法


C:\Users\ym500\Anaconda3\lib\site-packages\sklearn\cluster\birch.py:77: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  node1_dist, node2_dist = dist[[farthest_idx]]


result of harabaz_score:  65.89098437969326
result of silhouette_score:  0.06225485437395765
SpectralClustering算法
result of harabaz_score:  165.7773974035163
result of silhouette_score:  0.16336386750652035
聚类总情况数:  154
0
纠纷   119
被盗   80
1
挡道   576
称京   3
2
求助   523
称其   58
3
纠纷   301
求助   9
4
核实   208
询问   62
5
扰民   208
噪音   197
6
核实   180
朋友   18
第1次
K_Means算法
result of harabaz_score:  381.0454590527577
result of silhouette_score:  0.2032535555811582
Birch算法
result of harabaz_score:  166.75183348105733
result of silhouette_score:  0.07111624182502874
SpectralClustering算法
result of harabaz_score:  370.2941175317144
result of silhouette_score:  0.19640429946799023
聚类总情况数:  165
0
车主   72
派出所   63
1
挡道   575
称京   3
2
求助   401
称一   53
3
闹事   48
称一   39
4
纠纷   292
退货   2
5
打架   75
受伤   45
6
核实   249
电话   39
7
求助   246
接到   27
8
纠纷   200
发生   63
9
扰民   197
噪音   187
第2次
K_Means算法
result of harabaz_score:  924.5521505757255
result of silhouette_score:  0.20802976510400717
Birch算法
result of h

In [8]:
details = {}
labels_ = all_labels[0]
kind = df[1].tolist()
for i in range(len(data)):
    if labels_[i] not in details:
        details[labels_[i]] = [[data[i]], [kind[i]]]
    else:
        details[labels_[i]][0].append(data[i])
        details[labels_[i]][1].append(kind[i])
number = 0
for key, list in details.items():
    this_kind = 0
    print("第%d类" % key)
    for i in range(len(list[0])):
        this_kind = this_kind + 1
        print("属于", list[1][i], ' ', list[0][i])
    print(this_kind)
    print()
    print()
        

第9类
属于 new   称被台风困在水中，被困时间较长
属于 new   被台风困在水中，困了很长时间
属于 new   据说因为台风被水所困，所以长时间困着。
属于 new   他说他被台风困在水里，困了很长一段时间。
属于 new   说被台风困在水里被困了很久
属于 new   他说他被台风困在水里被困了很长时间
属于 new   因台风被困在水中，时间长而久
属于 new   他们说他被台风困在水里，困了很长时间。
属于 new   他说是被台风困在水里很长一段时间
属于 new   称被台风困在水中，被困时间较长
属于 new   称被台风困在水中，被困时间较长
属于 new   称台风导致停电
属于 new   台风造成停电
属于 new   台风导致停电。
属于 new   台风造成的停电
属于 new   台风引发了停电
属于 new   台风导致电力故障
属于 new   台风造成电力能量下降
属于 new   因台风而停电
属于 new   飓风导致停电
属于 new   称被台风困在候车大厅
属于 new   称因台风候车室被封锁
属于 new   称被困在台风中的候车室里
属于 new   他说被台风困在候车室中
属于 new   他们说被困在台风旁边的候车室里
属于 刑事类   称这款手机在126路公共汽车上被偷了
属于 刑事类   2016年1月4日10时许，胡某从建东二村这条路去往野水沟菜市场买完菜后，沿路往回家的方向走，走到234医院门口时，发现放在衣服包里的手机不见了。被盗手机为一部红色苹果7plus手机，价值人民币6388元。
属于 刑事类   电话被偷了。
属于 刑事类   称手机被盗。
属于 刑事类   称手机被盗。
属于 刑事类   2016年2月2日21时30分至2016年2月2日21时50分，莫某在回家的路上，步行到江北区建设东路兴业银行至江北区234医院A小区路段时，其钱包被盗。被盗钱包为一个黑色皮质钱包，钱包内有一张建行社保卡、一张新世纪提货卡，面值为1480元、两张银行卡、一张身份证、零钱81.5元。
属于 刑事类   称其手机被盗。
属于 刑事类   称手机被盗。
属于 刑事类   昨晚的电话被偷了
属于 刑事类   称昨晚手机在此处被盗
属于 刑事类   手机被盗
属于 刑事类   称手机被偷了。
属于 刑事

属于 公民求助   称一直接到骚扰电话，报警人说不出地址。
属于 公民求助   称多次接到骚扰电话。
属于 公民求助   举报贷款骚扰电话
属于 公民求助   称一直接到骚扰电话。
属于 公民求助   称接到诈骗电话，疑似身份信息泄露。
属于 公民求助   称收到多条验证消息，担心被骗。
属于 其他类   称被电话骚扰。
属于 公民求助   称家人以及报警人均收到一电话的骚扰，且收到多条骚扰短信。
属于 举报类   举报骚扰电话。
属于 公民求助   称被电话骚扰。
属于 公民求助   称经济纠纷，其家属收到骚扰短信。
属于 公民求助   称接到骚扰借款电话。
属于 公民求助   称被电话短信骚扰，且对方换着不同号码拨打。
属于 公民求助   称接到骚扰电话威胁报警人。
属于 刑事类   2016年1月7日,2016年1月7日,殷某被诈骗65000元,以贷款的名义,需要保险和支付保险费的能力和偿还还款的能力。在房地产市场上,中国银行(bank of China)的一个朋友jean的农业银行(agricultural al bank of jean)转移了15000元人民币,而other 5000元是支付银行银行卡Yin的余额。1万5万元通过一个微信号转移到别人账户。
属于 刑事类   2016年1月6日至2016年1月7日，殷某在重庆市江北区观音桥耐克佳苑3栋4－12被人以贷款需要缴纳保险费和证明还款能力的名义诈骗65000元人民币。其中有15000元人民币是通过朋友冉某的农业银行转账10000元，另外5000元是殷某的支付宝余额转账的，对方账号是中国工商银行银行卡（户名：罗某）。另外50000元是通过殷某的微信号向对方的微信号转账。
属于 刑事类   2016年1月6日,在重庆江北区,内环6 - 9被非法收取,费用为4853.8元。该公司被自己的微信转移了两倍,以欺诈提供的微信号,第一个3200元,第二个1653.8元。
属于 刑事类   2016年1月6日,从人那里说,买钱的钱和以海外保险的名义借的人要得到4853.8人的外币。通过自己的微信,将被提供为欺骗的微信号进行两次转账,是第一次3200元,第2回1653.8元。
属于 刑事类   2016年1月6日，杨某在重庆市江北区观音桥莲花池世纪内环6－9被人以贷款需要缴纳点子费和意外保险费的名义诈骗4853.8元

属于 纠纷   称看见一名出租车驾驶员被三名乘客打了。
属于 纠纷   称与人发生矛盾欲打架，暂无过激行为，需民警到场协调。
属于 纠纷   称看见四个人在打架，无持械。
属于 公民求助   称其朋友在此被打，具体情况不详，报过警，派出所到场做过笔录，现其朋友在234医院病房内，无人出面处理，感到不满，需联系民警。
属于 公民求助   回拨称一男一女醉酒打了人，现双方在场，暂不需120。
属于 其他类   称4、5个人在此闹事，持械不清，有人受伤，不用120。
属于 纠纷   称被人打了，现双方在场，不需120。
属于 其他类   称朋友被打。
属于 纠纷   称被打了。
属于 其他类   称五六人在此打架，之前有饮酒，未持械，暂无人受伤。
属于 其他类   称被打
属于 其他类   称一群女生打架，未持械，报过警民警未到。
属于 其他类   称看见7、8人打一女生，无持械。
属于 其他类   称朋友在此被打受伤，称民警放走了打人者，需民警联系报警人
属于 其他类   称有几名中国人打自己（报警人系韩国人）
属于 纠纷   称有10几人在此在此闹事，持酒瓶，有人伤，无需120 报警人说完：喂后挂断，回拨正在通话中。民警未到
属于 其他类   称昨日在此被打。
属于 纠纷   称被打。
属于 其他类   报警系现役军人，重复朋友被打伤一警，报警人想查监控
属于 其他类   称多人打架，不配合询问。
属于 其他类   称看见有十几人在此因醉酒准备打架，具体不清楚。
属于 其他类   称8人醉酒手持酒瓶在此打架，暂无人受伤。
属于 其他类   称被4、5人打，暂不需120.
属于 公民求助   称被打
属于 其他类   称被打。
属于 其他类   称自己被打。
属于 其他类   称4、5名客人打架，详情不清。
属于 其他类   称其被打，暂不需120.
属于 其他类   称此处有3，4人打自己 ，望联系。
属于 其他类   称7、8人喝醉酒在此打架，未持械，受伤情况不清。
属于 其他类   称看见10几人（90后）在此打架，未持械，无人受伤，现引起堵车，具体情况不清。
属于 其他类   称其朋友被出租车司机打伤，其朋友膝盖受伤，不需要120。
属于 其他类   称朋友被打，现报警人和朋友在234医院。
属于 纠纷   称几十个人打了报警人，报警人疑似头部受伤，请民警视情况

属于 刑事类   2016年3月10日凌晨1时许，报警人王某在重庆市江北区苏莲酒吧喝酒时手机被盗，被盗手机是一部磨砂黑苹果7PLUS手机，32G，2015年8月购买，购价6000元左右，手机能正常使用，手机右上角屏幕有裂痕，不清楚手机串号。现立为刑事案件。
属于 刑事类   称其物品被盗。
属于 刑事类   称电脑被盗，还被人威胁。
属于 刑事类   2016年4月11日凌晨1时许，报警人于某在重庆市江北区八街星空酒吧被人趁其不备盗窃其一部手机。被盗窃的是一部红色苹果7 plus手机，2015年9月以7100元购买，128GB内存，手机外面包有一个透明的塑料外壳。拟立刑事案件侦查。
属于 刑事类   手机被盗
属于 刑事类   称钱包被盗。
属于 刑事类   称锁被撬，丢失了8圈电线
属于 刑事类   称手机被盗
属于 刑事类   手机在此被盗。
属于 刑事类   称家中被盗。
属于 刑事类   称手机被盗。
属于 刑事类   称手机在此遗失。
属于 刑事类   2016年5月28日20时40分许，民警接警后到达重庆江北区梦想21公寓18－3，经了解，系2016年35月28日3时许至8时许报警人胡某将一台银色苹果笔记本（型号MAC PRO2015款，2015年在香港买成10000元）和一部黑色苹果7（128G，买成大概5000元左右）放在位于重庆江北区梦想21公寓18－3合租房的公共办公桌上充电，后发现东西不见了。房间门锁无撬动痕迹。报警人已来所制作询问笔录，你离刑事案件侦查
属于 刑事类   称手机丢失。南岸复：报案人称自己手机在江北区八街丢失。
属于 刑事类   称手机丢失。
属于 刑事类   称朋友手机被偷
属于 刑事类   2018年1月8日22时许，金某到我所报警称，2018年1月8日20时许，其在星空酒吧上班时将自己的身份证、农业银行卡、手机在上班时放在星空酒吧包房内。2018年1月8日7时许，报警人在网上购物后，发现银行卡内存款余额错误，遂查询银行交易明细，发现绑定在微信的银行卡内的钱分多次被转走3600元。报警人意识到被盗，遂报警。
属于 刑事类   称口红机里面的东西被盗。
属于 刑事类   称手机被盗
属于 刑事类   称钱包（身份证、银行卡、5000元现金）被偷
属于 刑事类   称银行卡在上述地址被盗刷2000元
属于 刑事类   手机被盗


属于 公民求助   渝C5M676占用车位。
属于 公民求助   称渝B1D586挡道。 称96009也联系不到车主，需民警再次通知。
属于 公民求助   渝CKY129挡道。 车未挪。
属于 公民求助   称要去乘飞机，粤BAR025挡道。
属于 110工作特殊需要   粤AE338U挡道。车未挪
属于 110工作特殊需要   粤AT225N挡道，车未挪开
属于 纠纷   称一车粤A2352P故意挡道起纠纷。
属于 110工作特殊需要   称有人在此闹事，车把车库堵了。
属于 公民求助   粤D065L5挡道，车主在场但拒绝挪车
属于 公民求助   粤ASK387挡道。 车未挪。
属于 公民求助   称粤B2784警挡道
属于 公民求助   粤D556T0挡道。事发地在花市附近久鼎花园消防通道，属于江北。
属于 110工作特殊需要   称粤APU727挡道。 称车主未到。
属于 公民求助   粤ASP163占用车位。
属于 110工作特殊需要   粤AL2726挡道。车未挪
属于 110工作特殊需要   称粤BBF653挡道，车未挪
属于 公民求助   称多辆盼达车占道停车，严重影响交通。
属于 公民求助   称两车挡道，无车牌。
属于 公民求助   称车主在此故意挡道，不肯挪车
属于 道路交通类   粤BWK002（哈弗）挡住120通道。
属于 110工作特殊需要   粤D763N3挡道，无法卸货。
属于 110工作特殊需要   称粤D633B7挡道。 车未挪。
属于 110工作特殊需要   称粤D633B7挡道。 车未挪。云B3G11N挡道。
属于 公民求助   云B3G11N挡道。车未挪
属于 110工作特殊需要   云B3G11N挡道。车未挪。车未挪。
属于 公民求助   称有大货车在此处卸货，导致交通不便。
属于 公民求助   称粤D3N389挡道。 车主未到。
属于 公民求助   粤AD51807（绿牌）挡道
属于 道路交通类   称多车乱停放，堵塞严重
属于 110工作特殊需要   称多车占道停放，导致交通堵塞。
属于 公民求助   粤BDJ496挡道，车未挪
属于 公民求助   粤B2764警 挡道
属于 110工作特殊需要   粤BAR838占车位
属于 公民求助   称川SＥＢ５６１挡道。
属于 公民求助   粤B93X07占停消防通道，请挪车

...

属于 公民求助   称装修噪音扰民，求助民警处理。
属于 公民求助   称此处音响声扰民，求助。
属于 治安类   称刚刚在此被网上约的女子骗了，求助民警。
属于 公民求助   银行卡被他人盗走后取了钱，求助
属于 治安类   称微信被骗1200元，求助
属于 纠纷   称债务纠纷，现欠债人发病120到场，债权人阻拦120离开，求助
属于 公民求助   称遭到小贷公司的骚扰，求助民警
属于 其他类   称女友李某数日都联系不上，担心其安全，报警人（杨某）现在江苏，求助
属于 其他类   称朋友（黄某）从5月15号开始一直联系不上，担心其安危，求助
属于 公民求助   称其老婆（刘某）离家出走一个月，现联系不上，求助。
属于 其他类   称租赁纠纷，求助民警处理。
属于 纠纷   称租赁纠纷，求助民警
属于 纠纷   称租赁纠纷，求助民警。
属于 纠纷   称在此处租住了两个月，到了租期后对方没有按照约定退押金，现一直联系不上对方，求助，报警人现在江苏。
273


第7类
属于 刑事类   称一外国朋友昨晚在此丢失手机，现定位手机在上清寺，求助
属于 纠纷   称擦挂。事发地在五红路，未过加油站，属江北。补充：对方驾驶员打报警人，求助。 称民警未到。
属于 公民求助   称农业银行转款（2000元）转错到他人账户，报警人现在黄泥磅万洋花市农业银行，求助。
属于 纠纷   称担保公司不退资料引发纠纷，双方协商不成，求助民警。
属于 纠纷   称公司其中一名股东私自将公章拿走到处行骗，现拒绝交还，求助。
属于 公民求助   报警人系滴滴车司机，称在歇马接到一名乘客，乘客在车上向报警人借款300元，现乘客未还钱，且乘客使用的滴滴账户是别人的，现联系不到乘客，求助民警。
属于 公民求助   称单车事故，此处掉落一不明物体，砸烂报警人后车窗玻璃，求助。
属于 其他类   称上周五车停在此处车窗被砸，求助民警调查监控。
属于 公民求助   称其车停此油箱被刺破，导致油漏到地面，存在安全隐患，求助民警。
属于 治安类   称川B2570P车子停在上述地址被钢珠枪打碎，情况不清，求助民警。
属于 治安类   称其车轮胎被划，报警人不在现场，求助民警调查处理。
属于 公民求助   报警人车辆（渝I3B797，吉利博瑞白色）于昨天晚上借给其同事（安迪，17358457671）后，对方至

属于 纠纷   称被打，请核实。
属于 其他类   称有人欲殴打自己，报警人系未成年人，请核实。
属于 纠纷   报警女子不配合询问，一直说在此处有人抢她项链，挂断电话回拨数次无人接听，请核实。
属于 其他类   称几人打架，其他情况不透露。补充民警未联系，请核实。 补充两人打架，不需120.
属于 其他类   称此处有人打架，不配合询问，请核实。民警未到。
属于 纠纷   称有五人打了自己，未持械，对方在场，暂不需120。 渝北核实，银紫薇KTV属江北区。
属于 其他类   称此处有6个男生打一女生，未携带工具，请核实
属于 公民求助   称报警人冲撞了对方，对方十余人欲打报警人，不理询问，具体不清，请核实。
属于 纠纷   称被打。报警人系一名女性，称民警十分钟必须到场，态度有些恶劣。请核实。
属于 纠纷   称被人用烟酒烫伤，请核实。
属于 纠纷   报警人称同事被打，未说清具体地址将电话挂断，请核实处理
属于 其他类   称有人打报警人，请核实。
属于 纠纷   称伤人，具体报警人不理询问，回拨多次被挂断，请核实。
属于 纠纷   称在此唱歌，被人打伤，报警人已离开，请核实。
属于 纠纷   称被打，具体情况不详，请核实。
属于 纠纷   报警人未说情况挂断，请核实。
属于 纠纷   称朋友在此被人捅了两刀，告120已知，请核实。 3:01询 3:06复：轻伤，已带回所
属于 其他类   称朋友被十几人打了，现暂未打架了，请核实。
属于 纠纷   报警人说完地址和我前男友后挂断，疑似有人抢夺报警人手机，请核实。
属于 公民求助   称看见3，4人在此打架，具体不详，请核实
属于 纠纷   称有人打自己，后电话中断，回拨仍挂断，请核实。
属于 纠纷   称看见7-8人围殴一对夫妻，未持械，受伤情况不清，请核实. 2:05询复：买水果纠纷，已妥处。
属于 纠纷   称有人（认识的人）要杀自己，报警人不配合询问，只让快来，请核实
属于 纠纷   报警人称有7、8人打架，有人持刀，请核实 5:52询;民警现场核实，系喝醉酒发生纠纷，无人持刀，无过激，告妥处。
属于 纠纷   电话中疑似有争吵声，需核实。
属于 纠纷   称有多人在此处打架，具体情况不详，请核实
属于 纠纷   报警人称打架，不配合询问，与人争吵，具体不清，请核实。
属于 纠纷   称有人在此打架，详

属于 其他类   称纠纷，对方称欲打自己父亲。
属于 纠纷   称因整形发生纠纷。
属于 纠纷   称住宿纠纷。
属于 其他类   称其与麻将馆老板发生纠纷，地址不清。 称民警未到，望联系。
属于 公民求助   称装修纠纷。
属于 纠纷   称快递纠纷。
属于 纠纷   称搬家发生纠纷
属于 纠纷   称装修纠纷。
属于 110工作特殊需要   外卖纠纷。
属于 纠纷   开锁纠纷。
属于 纠纷   称开锁纠纷。
属于 110工作特殊需要   称开锁纠纷。民警未到
属于 纠纷   称购房纠纷。电话联系报警人，地点在江北区八街附近不属我辖区。
属于 纠纷   称开锁费纠纷。
属于 公民求助   称工伤纠纷
属于 公民求助   称工伤纠纷
属于 公民求助   称清洁工工伤纠纷
属于 公民求助   称清洁工受伤，现纠纷。
属于 纠纷   称工人做工时受伤，工伤纠纷
属于 纠纷   称装修时受伤，现工伤纠纷
属于 纠纷   称其母亲做清洁时受伤，现工伤纠纷
属于 纠纷   称工作时摔伤，赔偿纠纷
属于 纠纷   称工伤纠纷。
属于 纠纷   工伤纠纷
属于 纠纷   称疑似纠纷。
属于 纠纷   称租赁纠纷，合同未到期，房东不让自己进屋。
属于 纠纷   系托管公司，称租赁纠纷。
属于 纠纷   称与保险公司有纠纷，保险公司诈骗之类，具体未理会询问。
属于 纠纷   称门面转让发生纠纷。
属于 其他类   称路过看见有人醉酒闹事砸店，疑似有人被刀割到动脉，具体不清，请核实，告120 2:52告江北 3:18江北复：系情侣纠纷
属于 纠纷   称感情纠纷，被男朋友打了。
属于 纠纷   称感情纠纷
属于 其他类   称自己被男友殴打，感情纠纷。
属于 纠纷   称与男朋友发生纠纷后打自己，现担心其安全，需民警联系
属于 其他类   称感情纠纷。
属于 公民求助   称因感情纠纷，其身份证、银行卡等被前男友拿走，至今未归还。
属于 纠纷   称此处三人左右持刀在此打架，暂无人伤，请核实。 23:13复 感情纠纷。
属于 纠纷   称纠纷，对方欲喊人打架，请核实。
属于 纠纷   称感情纠纷。
属于 纠纷   称感情纠纷，称现被对方殴打
属于 其他类   称感情纠纷
属于 纠纷   称感情纠纷，男朋友虐待报警人（报警人自称是孕妇）
属于 其他类   称感情纠纷。
属于 纠纷   

属于 纠纷   称纠纷。
属于 纠纷   纠纷
属于 纠纷   称纠纷。
属于 纠纷   踢车纠纷
属于 纠纷   称纠纷。
属于 其他类   称纠纷。
属于 纠纷   称纠纷
属于 其他类   称纠纷。 称民警未联系。
属于 其他类   纠纷。
属于 其他类   称纠纷。
属于 纠纷   称纠纷。
属于 其他类   称纠纷
属于 其他类   纠纷
属于 纠纷   称纠纷。
属于 公民求助   称纠纷。
属于 其他类   称纠纷。
属于 其他类   纠纷，5、6人阻碍办公
属于 纠纷   称纠纷。
属于 纠纷   称纠纷。
属于 其他类   称行车纠纷。
属于 其他类   称纠纷。
属于 其他类   称纠纷。
属于 公民求助   称纠纷。
属于 其他类   称纠纷。
属于 公民求助   纠纷。
属于 110工作特殊需要   称纠纷
属于 110工作特殊需要   称锁车纠纷。
属于 110工作特殊需要   纠纷
属于 110工作特殊需要   纠纷。
属于 110工作特殊需要   称纠纷
属于 110工作特殊需要   称纠纷。
属于 110工作特殊需要   纠纷。
属于 110工作特殊需要   称纠纷。
属于 公民求助   称纠纷，一车故意堵在该处。
属于 其他类   称纠纷。
属于 纠纷   纠纷
属于 纠纷   称纠纷
属于 其他类   纠纷
属于 其他类   称纠纷。需联系。
属于 纠纷   称纠纷
属于 纠纷   纠纷
属于 其他类   纠纷
属于 其他类   称纠纷。
属于 其他类   纠纷。
属于 公民求助   称纠纷
属于 纠纷   称纠纷
属于 公民求助   称纠纷。
属于 纠纷   称纠纷。
属于 纠纷   称纠纷。
属于 其他类   称纠纷。
属于 纠纷   称纠纷。
属于 其他类   称纠纷。
属于 其他类   称货运纠纷。
属于 纠纷   称纠纷
属于 其他类   称纠纷
属于 其他类   称纠纷。
属于 其他类   称纠纷。
属于 其他类   称纠纷。
属于 纠纷   称纠纷
属于 纠纷   退钱纠纷
属于 纠纷   称纠纷。
属于 其他类   称纠纷，在此办会员充值一万元，现不退了
属于 其他类   称纠纷。
属于 纠纷   称纠纷。
属于 公民求助   称纠纷。
属于 纠纷   称纠纷。
属于 其他类   称纠纷
属于 其他类   称纠纷。

属于 公民求助   称鄂A58Y38挡道
属于 公民求助   粤AY561V挡道
属于 公民求助   称粤A2530C挡道。
属于 公民求助   浙B13AB9挡道
属于 公民求助   浙B13AB9挡道。
属于 公民求助   粤A820CU挡道。
属于 公民求助   粤BV3661挡道
属于 公民求助   粤A185D3挡道。
属于 公民求助   粤FHE348挡道
属于 公民求助   粤BWD921挡道
属于 公民求助   粤D308R1挡道
属于 公民求助   粤BWD921挡道
属于 公民求助   粤D69Q13挡道
属于 公民求助   粤BLQ369挡道
属于 110工作特殊需要   粤D69Q13挡道
属于 公民求助   粤DBT714挡道。
属于 110工作特殊需要   粤DDT714挡道。
属于 公民求助   粤ABF424，粤AH1012挡道
属于 公民求助   粤AMC999挡道。
属于 公民求助   粤AXE293挡道
属于 公民求助   粤AB600P挡道。
属于 公民求助   粤A23T38挡道。
属于 公民求助   粤AQ056W挡道
属于 公民求助   称粤D61337挡道。
属于 公民求助   粤LBP332挡道
属于 公民求助   粤BDJ496挡道
属于 公民求助   粤AY193M挡道
属于 公民求助   粤A007G8挡道
属于 公民求助   称粤ASR193挡道。
属于 公民求助   粤BLQ369挡道。
属于 110工作特殊需要   粤AGL713挡道
属于 公民求助   粤AUU924挡道。
属于 公民求助   粤D67H17挡道
属于 公民求助   粤DBG202挡道。
属于 公民求助   称粤A9913U挡道
属于 公民求助   鄂A3AR00挡道
属于 公民求助   粤BFG549挡道
属于 道路交通类   称粤B0Z188挡道。
属于 道路交通类   称粤BXY180挡道。
属于 公民求助   称粤BBW713挡道。
属于 公民求助   称粤D3Q039挡道。
属于 道路交通类   称京OR9J06挡道。
属于 110工作特殊需要   称京OR9J06挡道
属于 公民求助   川BE6F06挡道
属于 公民求助   粤AE033Z挡道。
属于 公民求助   粤DAC673挡道。
属于 公民求助   称渝H3B79